In [33]:
from bs4 import BeautifulSoup as bs
import lxml.html as lh
from lxml.html import builder as E
from bs4 import BeautifulSoup
import requests
import json
import re
import csv
import pandas as pd

In [34]:
with open ("usdot_10+_4.txt", "r") as myfile:
    data = myfile.read()
data=data.replace("[","").replace("]","").replace("'","").replace('"',"")
_all_usdots = data.split(',')
all_usdots=[]
for element in _all_usdots:
    all_usdots.append(element.strip())

In [35]:
len(all_usdots)

88169

each company

In [36]:
def get_company_page(usdot_no):
    cookies = {
        'X-KEY': 'eyJpdiI6InVVc01BR3dvYm8xNmEyYko1NnYwTGc9PSIsInZhbHVlIjoiVXBpZDlCbThzdkNDWkcyVityU2UzMXUwaEpOV1o3MG13T0o3Q3o3U0xkdWFnVkdnSll0eHRaZGltV1QwRk1vdUhKVVBTbkkvcEZuV3IzN2VjSXA5OFBENVNCNUlIcHBFQThVbHd5TityRkM3ZURmUWpMWWF5RzU2R0cvTXdFd2M5bUtacVpWUVJBMTNISEM0OGN5bkhvczJDNnlucUJvYWpDaHV2ZS9FWnRZZExXdExHVkFWajFCZDRGU0JMeXl5SG56RmE2bVdtYlpMMFVrWnUydExXa2xRNVVpT1ZpUHhLT3VneXFtbFE4bFl2em1IOVpONlZySExnL0xJaThSVyIsIm1hYyI6IjI1NDI1YWEzZmUwMzRhNDdjODQ3MDhhNDA1YjZiMDBhYzJhMzRiYjU5OTk4YmUxN2NhMzg1OWU0ZDg0NTQ2ZDQifQ%3D%3D',
        '_ga': 'GA1.1.1581905416.1655785741',
        '__gads': 'ID=99f07c8c1a291d4d-2257f1a267d40030:T=1655785920:RT=1655785920:S=ALNI_MYlSRLddQtXSEyb4lHHQsOjsdUeUg',
        '__gpi': 'UID=000006d14ebceca5:T=1655785920:RT=1655807753:S=ALNI_MZh0MOvwE_5nwCwuuuhUqb7KFLpkQ',
        '_ga_2CX7JCZ1RS': 'GS1.1.1655807743.7.1.1655807957.0',
    }

    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Language': 'en-US,en;q=0.9,ru;q=0.8',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        # Requests sorts cookies= alphabetically
        # 'Cookie': 'X-KEY=eyJpdiI6InVVc01BR3dvYm8xNmEyYko1NnYwTGc9PSIsInZhbHVlIjoiVXBpZDlCbThzdkNDWkcyVityU2UzMXUwaEpOV1o3MG13T0o3Q3o3U0xkdWFnVkdnSll0eHRaZGltV1QwRk1vdUhKVVBTbkkvcEZuV3IzN2VjSXA5OFBENVNCNUlIcHBFQThVbHd5TityRkM3ZURmUWpMWWF5RzU2R0cvTXdFd2M5bUtacVpWUVJBMTNISEM0OGN5bkhvczJDNnlucUJvYWpDaHV2ZS9FWnRZZExXdExHVkFWajFCZDRGU0JMeXl5SG56RmE2bVdtYlpMMFVrWnUydExXa2xRNVVpT1ZpUHhLT3VneXFtbFE4bFl2em1IOVpONlZySExnL0xJaThSVyIsIm1hYyI6IjI1NDI1YWEzZmUwMzRhNDdjODQ3MDhhNDA1YjZiMDBhYzJhMzRiYjU5OTk4YmUxN2NhMzg1OWU0ZDg0NTQ2ZDQifQ%3D%3D; _ga=GA1.1.1581905416.1655785741; __gads=ID=99f07c8c1a291d4d-2257f1a267d40030:T=1655785920:RT=1655785920:S=ALNI_MYlSRLddQtXSEyb4lHHQsOjsdUeUg; __gpi=UID=000006d14ebceca5:T=1655785920:RT=1655807753:S=ALNI_MZh0MOvwE_5nwCwuuuhUqb7KFLpkQ; _ga_2CX7JCZ1RS=GS1.1.1655807743.7.1.1655807957.0',
        'Pragma': 'no-cache',
        'Referer': 'https://usdot411.com/user/search/advanced',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    return requests.get('https://usdot411.com/'+str(usdot_no), cookies=cookies, headers=headers)

In [37]:
def get_clean(usdot_no):
    company_page=get_company_page(usdot_no).text
    soup = bs(company_page)               
    prettyHTML = soup.prettify()
    cleantext = BeautifulSoup(prettyHTML, "lxml").text.strip('\n')
    cleantext=' '.join(cleantext.split())
    
    return cleantext

In [38]:
all_records=[]

In [39]:
for number in all_usdots:
    data={}
    try:
      content=get_clean(number)
      data["USDOT"]=number
      data["content"]=content
      all_records.append(data)
    except:
        print("ERROR in:"+str(number))

ERROR in:179484
ERROR in:471799
ERROR in:471949
ERROR in:472115
ERROR in:472634
ERROR in:473195
ERROR in:473668
ERROR in:473868
ERROR in:473901
ERROR in:473937
ERROR in:474521
ERROR in:474876
ERROR in:475985
ERROR in:476330
ERROR in:477041
ERROR in:477323
ERROR in:477364
ERROR in:477457
ERROR in:477667
ERROR in:477860
ERROR in:477871
ERROR in:477896
ERROR in:478488
ERROR in:478713
ERROR in:478847
ERROR in:478860
ERROR in:478862
ERROR in:479045
ERROR in:479802
ERROR in:480096
ERROR in:480541
ERROR in:480879
ERROR in:481053
ERROR in:481221
ERROR in:482690
ERROR in:482751
ERROR in:483147
ERROR in:483799
ERROR in:384727
ERROR in:312536
ERROR in:312719
ERROR in:312851
ERROR in:313058
ERROR in:313211
ERROR in:313502
ERROR in:313726
ERROR in:313800
ERROR in:313973
ERROR in:314006
ERROR in:314048
ERROR in:314122
ERROR in:314243
ERROR in:314293
ERROR in:314351
ERROR in:314548
ERROR in:314553
ERROR in:314826
ERROR in:314833
ERROR in:314844
ERROR in:314943
ERROR in:315036
ERROR in:315064
ERROR in

In [45]:
all_records[:2]

[{'USDOT': '883',
  'content': 'NORTHWEST PIPE FITTINGS INC (USDOT 883) US DOT 411 Search Monitoring Analytics NORTHWEST PIPE FITTINGS INC USDOT 883 Add Review No recent alerts Data 09/01/20 Monitor Get alerts Status HazMat Insurance Compliance Government Safety Rating Reliance Company Operating status ACTIVE DBA Entity type Carrier Operations Interstate Classification Private Property Docket # DUNS # 7499718 SCAC MCS-150 form date 11/22/2019 MCS-150 Mileage/Year 233,290 (2018) Fleet Details Power units 11 Autonomous units Electric units Drivers 7 SmartWay Not a SmartWay Partner CARB TRU Not 100% compliant Contacts Business address 2309 W OMAHA ST RAPID CITY, SD 57702-2462 Phone (605) 342-5587 Fax (605) 342-0673 Email lblack@northwestpipe.com Web www.northwestpipe.com Mailing address P O BOX 920 RAPID CITY, SD 57709-0920 Representative SCOTT BARBOUR DOUG JOHNSON Commodities Building Materials HazMat Hazardous Materials Safety Permit (HMSP) Status NO Since January 1, 2005, intrastate, i

In [46]:
df = pd.DataFrame.from_dict(all_records) 

In [47]:
df.shape

(29594, 2)

In [48]:
df.head()

,USDOT,content
0,883,NORTHWEST PIPE FITTINGS INC (USDOT 883) US DOT...
1,1596,FOLTZ LITTER INC (USDOT 1596) US DOT 411 Searc...
2,2340,KORTH TRUCKING INC (USDOT 2340) US DOT 411 Sea...
3,2824,AMERICAN BLOCK COMPANY INC (USDOT 2824) US DOT...
4,4400,CAMPBELL TRACTOR & IMPLEMENT INC (USDOT 4400) ...


In [49]:
df.to_csv (r'../raw_data/raw_1.csv', index = False, header=True)